In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [0]:
import numpy as np
np.random.seed(7)

In [0]:
from keras.datasets import mnist
(x_train, _), (x_test, _) = mnist.load_data()

In [0]:
input_size = np.prod(x_train.shape[1:])
print('input size -',input_size)

In [0]:
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

In [0]:
x_train = x_train.reshape((len(x_train), input_size))
x_test = x_test.reshape((len(x_test), input_size))

In [0]:
from keras.layers import Input, Dense
from keras.models import Model

In [0]:
embedding_size = 32
input_image = Input(shape=(input_size,))

encoded_image = Dense(embedding_size, activation='relu')(input_image)
decoded_image = Dense(input_size, activation='sigmoid')(encoded_image)

In [0]:
autoencoder = Model(input_image, decoded_image)
autoencoder.summary()

In [0]:
encoder = Model(input_image, encoded_image)
encoder.summary()

In [0]:
encoded_input = Input(shape=(embedding_size,))
decoder_layer = autoencoder.layers[-1]
decoder = Model(encoded_input, decoder_layer(encoded_input))
decoder.summary()

In [0]:
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

In [0]:
autoencoder.fit(x_train, x_train,
                epochs=50,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, x_test))

In [0]:
encoded_images = encoder.predict(x_test)
decoded_images = decoder.predict(encoded_images)

In [0]:
decoded_images = autoencoder.predict(x_test)

In [0]:
import matplotlib.pyplot as plt

n = 10  # how many digits we will display
plt.figure(figsize=(20, 4))
for i in range(n):
    # display original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_images[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()